Requirements:
- [Get started](../01-Day_1/02-Get_started.ipynb)
- [Data Management](../01-Day_1/04-Data_management.ipynb)

# Loop tiling

Nested loops often reuse the same data across their iterations and keeping the working set inside the caches can improve performance.
Tiling is a partitioning method of the loops into blocks. It reorders the loops so that each block will repeatedly hit the cache.
A first usage restriction will thus be on the loops' nature itself: not all loops can benefit from tiling, only the ones that will reuse data while showing a poor data locality, thus leading to frequent cache misses.

<img src="../../pictures/tiles.png"/>

OpenACC allows to improve data locality inside loops with the dedicated _tile_ clause. 
It specifies the compiler to split each loop in the nest into 2 loops, with an outer set of tile loops and an inner set of element loops.

## Syntax

The tile clause may appear with the _loop_ directive for nested loops.
For _N_ nested loops, the tile clause can take _N_ arguments. The first one being the size of the inner loop of the nest, the last one being the size of the outer loop.

```fortran
!$acc loop tile(32,32)
do i = 1, size_i
    do j = 1, size_j
        ! A Fabulous calculation
    enddo
enddo
```

## Restrictions

- the tile size (corresponding to the product of the arguments of the tile clause) can be up to 1024
- for better performance the size for the inner loop is a power of 2 (best with 32 to fit a cuda warp)
- if the vector clause is specified, it is then applied to the element loop
- if the gang clause is specified, it is then applied to the tile loop
- the worker clause is applied to the element loop only if the vector clause is not specified

## Example

In the following example tiling is used to transpose a matrix.

In [ ]:
%%idrrun -a --time
!!  examples/Fortran/Loop_tiling_example.f90
program tiles_optimization
    use ISO_FORTRAN_ENV, only : INT32, REAL64
    implicit none

    integer(kind=INT32 ), parameter        :: nx = 10000, ny = 10000 ! put nx, ny = 20000 on V100-32
    real   (kind=REAL64), dimension(nx,ny) :: A
    real   (kind=REAL64), dimension(ny,nx) :: B
    integer(kind=INT32 )                   :: i, j

    !$acc data copyout(A(:,:), B(:,:))
    
    !$acc parallel loop
    do j = 1, ny
        do i = 1, nx
            A(i,j) = i + (j-1)*ny
        enddo
    enddo    

    !$acc parallel loop tile(32,32)
    do j = 1, nx
        do i = 1, ny 
            B(i,j) = A(j,i)
        enddo
    enddo
    !$acc end data

    write(0,'(a2)') 'A:'
    do i = 1, 5
        write(0,"(5f20.8)") A(1:5,i)
    enddo
   
    write(0,'(a2)') 'B:'
    do i = 1, 5
        write(0,"(5f20.8)") B(1:5,i)
    enddo

end program tiles_optimization      

## Exercise

In this exercise, you will try to accelerate the numerical resolution of the 2D Laplace's equation with tiles. You can see that tiles parameter should be chosen wisely in order not to deteriorate performance.

In [ ]:
%%idrrun -a
!!  examples/Fortran/Loop_tiling_exercise.f90
program laplace2d 
use ISO_FORTRAN_ENV, only : real64, int32
!use openacc
implicit none

! Calculated solution for (E,B) fields
real   (kind=real64), dimension(:,:), allocatable :: T, T_new
! Dimension of the system
integer(kind=int32 )                              :: nx, ny ! number of points

integer(kind=int32 )                              :: i, j, it
real   (kind=real64)                              :: erreur

nx = 20000 !30000
ny = 10000 !30000

allocate(T(nx,ny),T_new(nx,ny))

! initial conditions
do j=2,ny-1
    do i=2,nx-1
        T(i,j)     = 0.0_real64
        T_new(i,j) = 0.0_real64
    enddo
enddo
!
do i=1,nx
    T(i, 1) = 100.0_real64
    T(i,ny) = 0.0_real64
enddo
!
do i=1,ny
    T(1 ,i) = 0.0_real64
    T(nx,i) = 0.0_real64
enddo

! add acc directive
do it = 1, 10000
  erreur = 0.0_real64
  !add acc directive
  do j= 2,ny-1
      do i= 2,nx-1
         T_new(i,j) =  0.25_real64*(T(i+1,j)+T(i-1,j) + &
                                    T(i,j+1)+T(i,j-1))
         erreur = max(erreur, abs(T_new(i,j) - T(i,j)))
      enddo
  enddo

  if (mod(it,100) .eq. 0) print *, "iteration: ",it," erreur: ",erreur

  !add acc directive
  do j= 2,ny-1
  do i= 2,nx-1
     T(i,j) =  T_new(i,j)
  enddo
  enddo  
enddo


deallocate(T, T_new)

end program laplace2d

## Solution

In [ ]:
%%idrrun -a
!!  examples/Fortran/Loop_tiling_solution.f90
program laplace2d 
use ISO_FORTRAN_ENV, only : real64, int32
!use openacc
implicit none

! Calculated solution for (E,B) fields
real   (kind=real64), dimension(:,:), allocatable :: T, T_new
! Dimension of the system
integer(kind=int32 )                              :: nx, ny ! number of points

integer(kind=int32 )                              :: i, j, it
real   (kind=real64)                              :: erreur

nx = 20000 !30000
ny = 10000 !30000

allocate(T(nx,ny),T_new(nx,ny))

! initial conditions
do j=2,ny-1
    do i=2,nx-1
        T(i,j)     = 0.0_real64
        T_new(i,j) = 0.0_real64
    enddo
enddo
!
do i=1,nx
    T(i, 1) = 100.0_real64
    T(i,ny) = 0.0_real64
enddo
!
do i=1,ny
    T(1 ,i) = 0.0_real64
    T(nx,i) = 0.0_real64
enddo

!$acc data copy(T) create(T_new)
do it = 1, 10000
  erreur = 0.0_real64
  !$acc parallel loop tile(32,32) reduction(max:erreur)
  do j= 2,ny-1
      do i= 2,nx-1
         T_new(i,j) =  0.25_real64*(T(i+1,j)+T(i-1,j) + &
                                    T(i,j+1)+T(i,j-1))
         erreur = max(erreur, abs(T_new(i,j) - T(i,j)))
      enddo
  enddo

  if (mod(it,100) .eq. 0) print *, "iteration: ",it," erreur: ",erreur

  !$acc parallel loop tile(32,32)
  do j= 2,ny-1
  do i= 2,nx-1
     T(i,j) =  T_new(i,j)
  enddo
  enddo  
enddo
!$acc end data

deallocate(T, T_new)

end program laplace2d